# Анализа таблиц в Python

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from itertools import combinations, chain, product, starmap
import graphviz
import networkx as nx
import toolz
from collections import Counter, abc
from operator import itemgetter

In [2]:
rng = np.random.default_rng(42)

In [3]:
def get_candidates(df, n):
    rv = []
    for cols in combinations(df.columns, n):
        cols = list(cols)
        if df.shape[0] == df[cols].drop_duplicates().shape[0]:
            rv.append(cols)
    return rv

In [4]:
def is_candidate(df, cols):
    if not isinstance(cols, (list, pd.Index)):
        cols = [cols]
    if df.shape[0] == df[cols].drop_duplicates().shape[0]:
        return True
    else:
        return False

In [5]:
def fd(df, l, r):
    if not isinstance(l, list):
        l = [l]
    if not isinstance(r, list):
        r = [r]
    
    df = df[{*l, *r}].drop_duplicates().dropna()
    n = df.shape[0]
    fd_l = n == df[l].drop_duplicates().shape[0]
    fd_r = n == df[r].drop_duplicates().shape[0]
    
    return fd_l, fd_r 

In [6]:
def get_relation_analysis(df, l, r):
    if not isinstance(l, list):
        l = [l]
    if not isinstance(r, list):
        r = [r]
    
    df = df[[*l, *r]].drop_duplicates().dropna()
    
    l_dict = dict(sorted(Counter(df[l].value_counts().values).items(), key=itemgetter(0), reverse=False))
    r_dict = dict(sorted(Counter(df[r].value_counts().values).items(), key=itemgetter(0), reverse=False))
    
    cart = sum(l_dict.values()) * sum(r_dict.values())
    links = sum(k * v for k, v in l_dict.items())
    density = links / cart
    
    l_metric =  l_dict.get(1, 0) / sum(l_dict.values())
    r_metric =  r_dict.get(1, 0) / sum(r_dict.values())
    
    return l_dict, r_dict, round(l_metric, 1), round(r_metric, 1), round(density, 3)

In [7]:
# определяем все FD (наборы из 1 колонки)

def get_all_df_len2(df):
    l = []
    for cols in combinations(df.columns, 2):
        fd_rv = fd(df, cols[0], cols[1])
        if fd_rv[0] == True:
            l.append((cols[0], cols[1]))
        if fd_rv[1] == True:
            l.append((cols[1], cols[0]))
    return l

In [8]:
def unique_drop(df):
    df = df.drop_duplicates().copy()
    total_rows = df.shape[0]
    d = {}
    for col in df.columns:
        d[col] = total_rows - df[df.columns.difference([col])].drop_duplicates().shape[0]
    return d

In [9]:
def data_model(df):
    return df.groupby(df.isna().applymap(lambda el: el*1).agg(tuple, axis=1)).size()

In [10]:
# check_2NF

# часть ключа не может определять неключевой столбец
# предполагается что ключ один, он составной
# только fd длиной один анализируем

In [11]:
# check_3NF

# между неключевыми столбцами не может быть fd

In [12]:
def classic_blanks(df, l, r):
    df = df[[l, r]].drop_duplicates()

    l_blank_opposite = set(df[df[l].isna()][r].values)
    r_one_degree = set(toolz.dicttoolz.valfilter(lambda v: v == 1, df[r].value_counts().to_dict()).keys())
    l_blank_type = r_one_degree > l_blank_opposite if l_blank_opposite else 'no blank'

    r_blank_opposite = set(df[df[r].isna()][l].values)
    l_one_degree = set(toolz.dicttoolz.valfilter(lambda v: v == 1, df[l].value_counts().to_dict()).keys())
    r_blank_type = l_one_degree > r_blank_opposite if r_blank_opposite else 'no blank'

    return l_blank_type, r_blank_type

# Обзорный анализ таблицы

In [13]:
# https://www.kaggle.com/datasets/ad043santhoshs/sales-domain

In [14]:
# читаем таблицу

for p in Path.cwd().iterdir():
    if p.name.endswith('.csv'):
        globals()[p.stem] = pd.read_csv(p)

In [70]:
df = Sales_domain.copy()

In [16]:
# таблица влезет в Excel :)
# Общее количество строк и столбцов на листе 1 048 576 строк и 16 384 столбца

f'{df.shape[0]:_}', df.shape[1]

('1_048_575', 20)

In [17]:
# посмотрим на табилцу

df.sample(3)

,Date,product_code,customer_code,sold_quantity,fiscal_year,division,segment,category,product,variant,customer,platform,channel,market,sub_zone,region,gross_price,cost_year,manufacturing_cost,pre_invoice_discount_pct
272307,01-02-2020,A0118150103,90018106,14,2020,P & A,Peripherals,Internal HDD,AQ Dracula HDD � 3.5 Inch SATA 6 Gb/s 5400 RPM...,Premium,Flipkart,E-Commerce,Retailer,Spain,SE,EU,19.3630,2022,7.1831,0.2961
916812,01-02-2019,A0418150103,70008169,23,2019,P & A,Peripherals,Graphic Card,AQ Mforce Gen X,Standard 3,Atliq Exclusive,Brick & Mortar,Direct,Australia,ANZ,APAC,17.4436,2020,5.3448,0.0748
629615,01-04-2019,A0219150202,70023032,21,2019,P & A,Peripherals,Internal HDD,AQ WereWolf NAS Internal Hard Drive HDD � 8.89 cm,Plus,Atliq e Store,E-Commerce,Direct,Canada,NaN,NaN,20.8223,2020,7.0590,0.2911


In [18]:
# анализ трех рандомных строк
# получить общую картину о содержимом

with pd.option_context('max_colwidth', 20):
    display(df.sample(3).T)

,348754,560902,52244
Date,01-10-2021,01-02-2020,01-03-2020
product_code,A0118150103,A0219150201,A0118150101
customer_code,90015148,90015150,70023032
sold_quantity,249,7,15
fiscal_year,2022,2020,2020
division,P & A,P & A,P & A
segment,Peripherals,Peripherals,Peripherals
category,Internal HDD,Internal HDD,Internal HDD
product,AQ Dracula HDD �...,AQ WereWolf NAS ...,AQ Dracula HDD �...
variant,Premium,Standard,Standard


In [19]:
# смотрим на типы данных в теле

body_dtypes = set(chain.from_iterable(df.applymap(type).agg(set).values))
body_dtypes

{float, int, str}

In [20]:
# Неатомарные типы наследуют от Sized, Iterable, Container
# https://docs.python.org/3/library/collections.abc.html
    
body_dtypes
abc_class = [abc.Sized, abc.Iterable, abc.Container, abc.Hashable]
prod = list(product(body_dtypes, abc_class))
data = list(starmap(issubclass, prod))

(
pd
.DataFrame(data=data, index=pd.MultiIndex.from_tuples(prod))
.unstack()
.loc[:, pd.IndexSlice[0]]
)

,<class 'collections.abc.Sized'>,<class 'collections.abc.Iterable'>,<class 'collections.abc.Container'>,<class 'collections.abc.Hashable'>
<class 'str'>,True,True,True,True
<class 'float'>,False,False,False,True
<class 'int'>,False,False,False,True


In [21]:
# доказываем что строка - может выступать как неатомарный контейнер

s = 'some string'
for i in s:
    print(i, end='__')
print('\n')
print(list(reversed(s)))

s__o__m__e__ __s__t__r__i__n__g__

['g', 'n', 'i', 'r', 't', 's', ' ', 'e', 'm', 'o', 's']


In [22]:
%%time
# смотрим на уникальные значения в столбцах

for col in df.columns:
    print(f'{col:<25}', f'{len(list(df[col].unique())):<5}', ' >>> ', list(df[col].unique())[:5])

# есть столбцы с одним элементом
# в названии продукта проблемы с кодировкой

Date                      52     >>>  ['01-09-2017', '01-10-2017', '01-11-2017', '01-12-2017', '01-01-2018']
product_code              14     >>>  ['A0118150101', 'A0118150102', 'A0118150103', 'A0118150104', 'A0219150201']
customer_code             209    >>>  [70002017, 70002018, 70003181, 70003182, 70006157]
sold_quantity             1089   >>>  [51, 54, 30, 77, 53]
fiscal_year               5      >>>  [2018, 2019, 2020, 2021, 2022]
division                  1      >>>  ['P & A']
segment                   1      >>>  ['Peripherals']
category                  2      >>>  ['Internal HDD', 'Graphic Card']
product                   4      >>>  ['AQ Dracula HDD � 3.5 Inch SATA 6 Gb/s 5400 RPM 256 MB Cache', 'AQ WereWolf NAS Internal Hard Drive HDD � 8.89 cm', 'AQ Zion Saga', 'AQ Mforce Gen X']
variant                   8      >>>  ['Standard', 'Plus', 'Premium', 'Premium Plus', 'Standard 1']
customer                  75     >>>  ['Atliq Exclusive', 'Atliq e Store', 'Neptune', 'Synthetic'

In [23]:
df.nunique()

Date                          52
product_code                  14
customer_code                209
sold_quantity               1089
fiscal_year                    5
division                       1
segment                        1
category                       2
product                        4
variant                        8
customer                      75
platform                       2
channel                        3
market                        27
sub_zone                       6
region                         3
gross_price                   52
cost_year                      5
manufacturing_cost            51
pre_invoice_discount_pct     676
dtype: int64

In [24]:
# проверяем, что нет полных дубликатов

is_candidate(df, df.columns)

True

In [25]:
# желательно, чтобы индексы были униклаьные

df.index.is_unique, df.columns.is_unique

(True, True)

In [26]:
# есть ли бленки (логчески мы считаем бленками - None, np.nan, '')?

df.isna().any().any()

True

In [27]:
# распределение бленков по колонкам

df.isna().sum()

Date                             0
product_code                     0
customer_code                    0
sold_quantity                    0
fiscal_year                      0
division                         0
segment                          0
category                         0
product                          0
variant                          0
customer                         0
platform                         0
channel                          0
market                           0
sub_zone                    156972
region                      156972
gross_price                      0
cost_year                        0
manufacturing_cost               0
pre_invoice_discount_pct         0
dtype: int64

In [28]:
# обзор типов данных

df.dtypes

Date                         object
product_code                 object
customer_code                 int64
sold_quantity                 int64
fiscal_year                   int64
division                     object
segment                      object
category                     object
product                      object
variant                      object
customer                     object
platform                     object
channel                      object
market                       object
sub_zone                     object
region                       object
gross_price                 float64
cost_year                     int64
manufacturing_cost          float64
pre_invoice_discount_pct    float64
dtype: object

In [29]:
# обзор типов данных

df.convert_dtypes().dtypes

Date                         string
product_code                 string
customer_code                 Int64
sold_quantity                 Int64
fiscal_year                   Int64
division                     string
segment                      string
category                     string
product                      string
variant                      string
customer                     string
platform                     string
channel                      string
market                       string
sub_zone                     string
region                       string
gross_price                 Float64
cost_year                     Int64
manufacturing_cost          Float64
pre_invoice_discount_pct    Float64
dtype: object

In [30]:
# memory usage: 1.0 GB !!!

df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 20 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Date                      1048575 non-null  object 
 1   product_code              1048575 non-null  object 
 2   customer_code             1048575 non-null  int64  
 3   sold_quantity             1048575 non-null  int64  
 4   fiscal_year               1048575 non-null  int64  
 5   division                  1048575 non-null  object 
 6   segment                   1048575 non-null  object 
 7   category                  1048575 non-null  object 
 8   product                   1048575 non-null  object 
 9   variant                   1048575 non-null  object 
 10  customer                  1048575 non-null  object 
 11  platform                  1048575 non-null  object 
 12  channel                   1048575 non-null  object 
 13  market                    1

In [31]:
# распределение размера данных в долях по колонкам

(df.memory_usage(deep=True) / df.memory_usage(deep=True).sum()).round(2).sort_values(ascending=False)

product                     0.19
product_code                0.07
segment                     0.07
category                    0.07
customer                    0.07
platform                    0.07
variant                     0.06
division                    0.06
Date                        0.06
channel                     0.06
market                      0.06
region                      0.05
sub_zone                    0.05
manufacturing_cost          0.01
cost_year                   0.01
gross_price                 0.01
pre_invoice_discount_pct    0.01
fiscal_year                 0.01
sold_quantity               0.01
customer_code               0.01
Index                       0.00
dtype: float64

# Архитектурный анализ таблицы

In [32]:
df.head(5)

,Date,product_code,customer_code,sold_quantity,fiscal_year,division,segment,category,product,variant,customer,platform,channel,market,sub_zone,region,gross_price,cost_year,manufacturing_cost,pre_invoice_discount_pct
0,01-09-2017,A0118150101,70002017,51,2018,P & A,Peripherals,Internal HDD,AQ Dracula HDD � 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,Atliq Exclusive,Brick & Mortar,Direct,India,India,APAC,15.3952,2018,4.6190,0.0824
1,01-09-2017,A0118150101,70002017,51,2018,P & A,Peripherals,Internal HDD,AQ Dracula HDD � 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,Atliq Exclusive,Brick & Mortar,Direct,India,India,APAC,15.3952,2019,4.2033,0.0824
2,01-09-2017,A0118150101,70002017,51,2018,P & A,Peripherals,Internal HDD,AQ Dracula HDD � 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,Atliq Exclusive,Brick & Mortar,Direct,India,India,APAC,15.3952,2020,5.0207,0.0824
3,01-09-2017,A0118150101,70002017,51,2018,P & A,Peripherals,Internal HDD,AQ Dracula HDD � 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,Atliq Exclusive,Brick & Mortar,Direct,India,India,APAC,15.3952,2021,5.5172,0.0824
4,01-09-2017,A0118150101,70002017,51,2018,P & A,Peripherals,Internal HDD,AQ Dracula HDD � 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,Atliq Exclusive,Brick & Mortar,Direct,India,India,APAC,14.4392,2018,4.6190,0.0824


In [33]:
df.memory_usage(deep=True).sum()

1083801816

In [71]:
%%time
# заменяем все на IDшники
# архитектурный анализ не требует наличие конкретных данных, только ID
# это кстати упрощает

df = \
(df
 .apply(lambda s: s.factorize()[0])
 .astype('category')
 .apply(lambda s: 
                s.map(lambda el: np.nan if el == -1 else el)
       )
)

Wall time: 1.63 s


In [35]:
df.memory_usage(deep=True).sum()

38884923

In [36]:
df.head(5)

,Date,product_code,customer_code,sold_quantity,fiscal_year,division,segment,category,product,variant,customer,platform,channel,market,sub_zone,region,gross_price,cost_year,manufacturing_cost,pre_invoice_discount_pct
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,2,2,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,3,3,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1,0,0,0


In [72]:
one_elem_cols = ['division', 'segment']
unique_cols = [] # таких нет
expert_del_cols = [] # не будет делать экспертных суждений
# можно сделать предположения о ключах и о fd. Но мы просто сделаем анализ

In [73]:
%%time

# определяем все FD (источник и цель - по 1 колонке)

l = get_all_df_len2(df[df.columns.difference(one_elem_cols)])
l

Wall time: 10.2 s


[('Date', 'fiscal_year'),
 ('gross_price', 'category'),
 ('manufacturing_cost', 'category'),
 ('product', 'category'),
 ('product_code', 'category'),
 ('variant', 'category'),
 ('customer_code', 'channel'),
 ('manufacturing_cost', 'cost_year'),
 ('customer_code', 'customer'),
 ('customer', 'platform'),
 ('customer_code', 'market'),
 ('customer_code', 'platform'),
 ('customer_code', 'region'),
 ('customer_code', 'sub_zone'),
 ('gross_price', 'product'),
 ('gross_price', 'product_code'),
 ('gross_price', 'variant'),
 ('market', 'region'),
 ('market', 'sub_zone'),
 ('product_code', 'product'),
 ('product_code', 'variant'),
 ('sub_zone', 'region')]

In [75]:
# граф всех FD

g = graphviz.Digraph()
for edge in l:
    g.edge(f'{edge[0]}', f'{edge[1]}')
g.view()

'Digraph.gv.pdf'

In [76]:
# удаляем транзитивные зависимости через networkx, в одно действие
# https://en.wikipedia.org/wiki/Transitive_reduction

G = nx.DiGraph()
G.add_edges_from(l)
H = nx.transitive_reduction(G)
l_tred = list(H.edges)

In [77]:
# посмотрим, что удалили (все транзитивные зависимости)

set(l) - set(l_tred)

{('customer_code', 'platform'),
 ('customer_code', 'region'),
 ('customer_code', 'sub_zone'),
 ('gross_price', 'category'),
 ('gross_price', 'product'),
 ('gross_price', 'variant'),
 ('market', 'region'),
 ('product_code', 'category')}

In [80]:
# граф всех FD после tred - transitive reduction (более чистый граф, без лишней информации)

g = graphviz.Digraph()
for edge in l_tred:
    g.edge(f'{edge[0]}', f'{edge[1]}')
g.view()

'Digraph.gv.pdf'

In [43]:
# определяем колонки, которые можно не включать в проверку ключа изначально (networkx)
# если колонка определяется от другой, то на первом этапе можно взять только источник fd
# и ести источник по fd будет в ключе, то можно задаться вопросом по 'ослаблению', т.е. переходу по fd

G = nx.DiGraph()
G.add_edges_from(l_tred)
d = dict(G.in_degree)
d = toolz.dicttoolz.valfilter(lambda v: v != 0, d)
cols_fd_dep = list(d.keys())
len(cols_fd_dep)

12

In [44]:
# смотрим на колонки, которые можно убрать из анализа ключа

cols_fd_dep

['fiscal_year',
 'product_code',
 'cost_year',
 'category',
 'product',
 'variant',
 'market',
 'channel',
 'customer',
 'platform',
 'sub_zone',
 'region']

In [45]:
# составляем df для проверки

df_for_cand_find = df[df.columns.difference([*one_elem_cols, *cols_fd_dep])]

In [46]:
df_for_cand_find.shape

(1048575, 6)

In [47]:
df_for_cand_find.head(5)

,Date,customer_code,gross_price,manufacturing_cost,pre_invoice_discount_pct,sold_quantity
0,0,0,0,0,0,0
1,0,0,0,1,0,0
2,0,0,0,2,0,0
3,0,0,0,3,0,0
4,0,0,1,0,0,0


In [48]:
df_for_cand_find.sample(5)

,Date,customer_code,gross_price,manufacturing_cost,pre_invoice_discount_pct,sold_quantity
987336,37,118,44,42,469,9
778312,46,205,29,29,550,23
686701,43,60,25,25,477,9
88182,44,140,1,2,513,66
103509,6,23,5,5,23,41


In [49]:
%%time

get_candidates(df_for_cand_find, 1)

Wall time: 217 ms


[]

In [50]:
%%time

l = []
for n in range(1, 5+1):
    l.append(get_candidates(df_for_cand_find, n))

Wall time: 6.89 s


In [51]:
list(enumerate(l, start=1))

[(1, []),
 (2, []),
 (3, []),
 (4, [['Date', 'customer_code', 'gross_price', 'manufacturing_cost']]),
 (5,
  [['Date',
    'customer_code',
    'gross_price',
    'manufacturing_cost',
    'pre_invoice_discount_pct'],
   ['Date',
    'customer_code',
    'gross_price',
    'manufacturing_cost',
    'sold_quantity']])]

# Фокусный анализ fd

In [55]:
# переходим обратно к обычном представлению таблицы
df = Sales_domain.copy()

In [56]:
df.sample(3)

,Date,product_code,customer_code,sold_quantity,fiscal_year,division,segment,category,product,variant,customer,platform,channel,market,sub_zone,region,gross_price,cost_year,manufacturing_cost,pre_invoice_discount_pct
660125,01-05-2020,A0219150202,90021089,5,2020,P & A,Peripherals,Internal HDD,AQ WereWolf NAS Internal Hard Drive HDD � 8.89 cm,Plus,Atlas Stores,Brick & Mortar,Retailer,United Kingdom,NE,EU,25.1517,2022,7.2751,0.2089
508916,01-09-2021,A0118150104,70013125,89,2022,P & A,Peripherals,Internal HDD,AQ Dracula HDD � 3.5 Inch SATA 6 Gb/s 5400 RPM...,Premium Plus,Atliq Exclusive,Brick & Mortar,Direct,Italy,SE,EU,20.7734,2021,6.8199,0.0738
12822,01-07-2018,A0118150101,90022077,7,2018,P & A,Peripherals,Internal HDD,AQ Dracula HDD � 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,Radio Shack,Brick & Mortar,Retailer,USA,NaN,NaN,14.4392,2020,5.0207,0.2394


In [57]:
with pd.option_context('max_colwidth', 20):
    display(df.sample(3).T)

,853681,480848,933138
Date,01-11-2018,01-01-2021,01-07-2019
product_code,A0418150102,A0118150104,A0418150103
customer_code,90022072,90023029,90016171
sold_quantity,94,16,1
fiscal_year,2019,2021,2019
division,P & A,P & A,P & A
segment,Peripherals,Peripherals,Peripherals
category,Graphic Card,Internal HDD,Graphic Card
product,AQ Mforce Gen X,AQ Dracula HDD �...,AQ Mforce Gen X
variant,Standard 2,Premium Plus,Standard 3


In [58]:
# анализ связи между двумя колонками

get_relation_analysis(df, 'Date', 'fiscal_year')

({1: 52}, {4: 1, 12: 4}, 1.0, 0.0, 0.2)

In [59]:
get_relation_analysis(df, 'Date', 'cost_year')

({5: 52}, {52: 5}, 0.0, 0.0, 1.0)

In [60]:
get_relation_analysis(df, 'product_code', 'product')

({1: 14}, {3: 2, 4: 2}, 1.0, 0.0, 0.25)

In [61]:
get_relation_analysis(df, 'market', 'region')

({1: 25}, {4: 1, 10: 1, 11: 1}, 1.0, 0.0, 0.333)

In [62]:
get_relation_analysis(df, 'platform', 'channel')

({2: 1, 3: 1}, {1: 1, 2: 2}, 0.0, 0.3, 0.833)

In [63]:
get_relation_analysis(df, 'customer_code', 'customer')

({1: 209},
 {1: 19, 2: 39, 3: 11, 4: 2, 5: 1, 17: 1, 24: 1, 25: 1},
 1.0,
 0.3,
 0.013)

# Прочее

In [64]:
%%time
# анализ сочетаний бленк/элемент

data_model(df)

Wall time: 19.2 s


(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    891603
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0)    156972
dtype: int64

In [65]:
%%time
# обзор типов хвостов

for x, y in combinations(df.columns, 2):
    x_ = classic_blanks(df, x, y)[0]
    y_ = classic_blanks(df, x, y)[1]
    if x_ == False or y_ == False:
        print(x, y, x_, y_)

Date sub_zone no blank False
Date region no blank False
product_code sub_zone no blank False
product_code region no blank False
sold_quantity sub_zone no blank False
sold_quantity region no blank False
fiscal_year sub_zone no blank False
fiscal_year region no blank False
division sub_zone no blank False
division region no blank False
segment sub_zone no blank False
segment region no blank False
category sub_zone no blank False
category region no blank False
product sub_zone no blank False
product region no blank False
variant sub_zone no blank False
variant region no blank False
customer sub_zone no blank False
customer region no blank False
platform sub_zone no blank False
platform region no blank False
channel sub_zone no blank False
channel region no blank False
sub_zone region False False
sub_zone gross_price False no blank
sub_zone cost_year False no blank
sub_zone manufacturing_cost False no blank
sub_zone pre_invoice_discount_pct False no blank
region gross_price False no blank


In [66]:
%%time
# анализируем ключ длиной 4
# оценка значимости колонке в ключе

unique_drop(df[['Date', 'customer_code', 'gross_price', 'manufacturing_cost']])

Wall time: 1.02 s


{'Date': 1007628,
 'customer_code': 1040504,
 'gross_price': 798133,
 'manufacturing_cost': 793846}